In [171]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
import scipy.stats as stats
import itertools
from sklearn import metrics
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

## Spotify Calls

In [330]:
import requests
import json
import config

# retrieve access tokens

client_id = config.client_id
client_secret = config.client_secret

grant_type = 'client_credentials'

body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'

response=requests.post(url, data=body_params, auth = (client_id, client_secret)) 
response.text
token = eval(response.text)
print('Full Token Data')
print(token)
print('\n')
print('token needed:', token.get('access_token'))

Full Token Data
{'access_token': 'BQD4YA8a367zJ6hfwQEiQnmy5O9X1rxRiqupj2LDGY6USR9Q8TL9QjZjDTkW5FxReDonzShXGf-hTXZz_I8', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


token needed: BQD4YA8a367zJ6hfwQEiQnmy5O9X1rxRiqupj2LDGY6USR9Q8TL9QjZjDTkW5FxReDonzShXGf-hTXZz_I8


In [331]:
access_token = token.get('access_token')
headers = {'Authorization': 'Bearer '+ access_token}

## Genres Available in Spotify's API

Before choosing what genre's will be classifiers, I make a call to check which genre's are available.

In [41]:
# retreiving spotify's list of genres
avail_genres = requests.get('https://api.spotify.com/v1/recommendations/available-genre-seeds', headers=headers)
genres = avail_genres.json()
genres

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


## Genre Playlist IDs

After choosing 15 genres, use the playlist ID to retrieve the songs available in the the genre.

In [21]:
# make a list of genre playlist IDS and names. Then convert into a dictionary

list_of_genre_ids = ['7Mr3wEdKgaiAP4Cm2a6vda', 
                     '4MRGjKqlWuJZJ8XHOGcqkR', 
                     '37i9dQZF1DX4dyzvuaRJ0n',
                     '1o1HVRGIxwCcdSnNnZ69IC',
                     '37i9dQZF1DWWEJlAGA9gs0',
                     '0TCtFMz5lY6jTfusk66ZFj',
                     '3kTtdRE1CtRyRKdicfOGAR',
                     '30BUPgw52SWNm2ZWZZc86A',
                     '6Ph1K0QWCcEwYRr0VhVt6C',
                     '5khoF3ksobwfVwOazDqpqI',
                     '6mRRGF4klfgUzbD2ZKOCq0',
                     '37i9dQZF1DWZd79rJ6a7lp',
                     '37i9dQZF1DX0xLQsW8b5Zx',
                     '2SwjQPegrTTYaOsWQrwhMe'
                    ]

list_of_genre_playlists = ['hip hop playlist',
                           'post rock playlist',
                           'electronic playlist',
                           'detroit_techo playlist',
                           'classical playlist',
                           'disco playlist',
                           'electro indie pop playlist',
                           'industrial pop playlist',
                           'french playlist',
                           'spanish playlist',
                           'ska playlist',
                           'sleep playlist',
                           'rockabilly playlist',
                           '50s Hits playlist'
                           ]

genre_dict = {list_of_genre_playlists[i]: list_of_genre_ids[i] for i in range(len(list_of_genre_playlists))}
genre_dict

{'hip hop playlist': '7Mr3wEdKgaiAP4Cm2a6vda',
 'post rock playlist': '4MRGjKqlWuJZJ8XHOGcqkR',
 'electronic playlist': '37i9dQZF1DX4dyzvuaRJ0n',
 'detroit_techo playlist': '1o1HVRGIxwCcdSnNnZ69IC',
 'classical playlist': '37i9dQZF1DWWEJlAGA9gs0',
 'disco playlist': '0TCtFMz5lY6jTfusk66ZFj',
 'electro indie pop playlist': '3kTtdRE1CtRyRKdicfOGAR',
 'industrial pop playlist': '30BUPgw52SWNm2ZWZZc86A',
 'french playlist': '6Ph1K0QWCcEwYRr0VhVt6C',
 'spanish playlist': '5khoF3ksobwfVwOazDqpqI',
 'ska playlist': '6mRRGF4klfgUzbD2ZKOCq0',
 'sleep playlist': '37i9dQZF1DWZd79rJ6a7lp',
 'rockabilly playlist': '37i9dQZF1DX0xLQsW8b5Zx',
 '50s Hits playlist': '2SwjQPegrTTYaOsWQrwhMe'}

In [32]:
# call playlist ID by calling the genre name of the playlist 
genre_dict.get('hip hop playlist')

'7Mr3wEdKgaiAP4Cm2a6vda'

## Call Functions 1.1

In [12]:
# call spotify recieve playlist of songs as dictionary
def call_spotify(genre_ID):
    i = genre_ID
    base = 'https://api.spotify.com/v1/playlists/'
    end = '/tracks'
    r = requests.get(base + i + end, headers=headers)
    songs = r.json()
    return songs

In [31]:
# example of calling spotify to retrieve all songs in a genre playlist
call_spotify(genre_dict.get('hip hop playlist'))

{'href': 'https://api.spotify.com/v1/playlists/7Mr3wEdKgaiAP4Cm2a6vda/tracks?offset=0&limit=100',
 'items': [{'added_at': '2020-01-20T12:10:39Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/redmusiccompany'},
    'href': 'https://api.spotify.com/v1/users/redmusiccompany',
    'id': 'redmusiccompany',
    'type': 'user',
    'uri': 'spotify:user:redmusiccompany'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0vRvGUQVUjytro0xpb26bs'},
       'href': 'https://api.spotify.com/v1/artists/0vRvGUQVUjytro0xpb26bs',
       'id': '0vRvGUQVUjytro0xpb26bs',
       'name': 'Arizona Zervas',
       'type': 'artist',
       'uri': 'spotify:artist:0vRvGUQVUjytro0xpb26bs'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      

## Call Functions 1.2

In [33]:
# fetch id function
def fetch_ids(songs):
    ids = []
    for song in songs['items']:
        ids.append(song['track']['id'])
    return ids

# fetch audio feature links function
def audio_features_(data):
    base =  'https://api.spotify.com/v1/audio-features/'
    list_of_ids = []
    for i in data:
        list_of_ids.append(base + str(i))
    return list_of_ids

# retrieve dictionary of track feature data
def fetch_features(links):
    feature_data = []
    for i in links:
        x = requests.get(i, headers=headers)
        j = x.content
        feature_data.append(json.loads(j))
    return feature_data

In [34]:
# combing the three functions into one

# all functions together to pull song list in and return dataframe
def song_return_feature_df(songs):
    ids = fetch_ids(songs)
    links = audio_features_(ids)
    feature_list = fetch_features(links)
    df = pd.DataFrame(feature_list)
    return df

## Call Functions 1.3

In [35]:
# call spotify playlist, return features as dataframe
def call_spotify_return_feat_df(genre_id):
    songs = call_spotify(genre_id)
    df = song_return_feature_df(songs)
    return df

In [334]:
# example of full function. Calling for song features in hip hop playlist
french_df = call_spotify_return_feat_df(genre_dict.get('french playlist'))
french_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.219,0.16800,0,-16.267,1,0.0505,0.96500,0.106000,0.3730,0.548,69.091,audio_features,0IfKxVyiSQxRajCwEE4dqJ,spotify:track:0IfKxVyiSQxRajCwEE4dqJ,https://api.spotify.com/v1/tracks/0IfKxVyiSQxR...,https://api.spotify.com/v1/audio-analysis/0IfK...,201840,4
1,0.310,0.58900,6,-11.097,0,0.2730,0.72800,0.000000,0.9630,0.605,61.011,audio_features,6b1FdyzHhNxM5mC9xoMvUw,spotify:track:6b1FdyzHhNxM5mC9xoMvUw,https://api.spotify.com/v1/tracks/6b1FdyzHhNxM...,https://api.spotify.com/v1/audio-analysis/6b1F...,209173,4
2,0.534,0.56000,4,-7.565,1,0.0236,0.45100,0.000096,0.1100,0.599,89.885,audio_features,4sWGG5BEy2zYE81rFoHMBo,spotify:track:4sWGG5BEy2zYE81rFoHMBo,https://api.spotify.com/v1/tracks/4sWGG5BEy2zY...,https://api.spotify.com/v1/audio-analysis/4sWG...,275040,4
3,0.784,0.31900,2,-11.813,1,0.0810,0.85000,0.000000,0.1290,0.777,115.591,audio_features,6DadMrUGPSV3gxQIHJ56C8,spotify:track:6DadMrUGPSV3gxQIHJ56C8,https://api.spotify.com/v1/tracks/6DadMrUGPSV3...,https://api.spotify.com/v1/audio-analysis/6Dad...,178440,3
4,0.790,0.10200,11,-16.727,0,0.1220,0.94100,0.000000,0.1160,0.714,101.641,audio_features,6axtrF0lqCidyJAlxzX6F8,spotify:track:6axtrF0lqCidyJAlxzX6F8,https://api.spotify.com/v1/tracks/6axtrF0lqCid...,https://api.spotify.com/v1/audio-analysis/6axt...,182360,4
5,0.372,0.00632,1,-31.050,1,0.0462,0.98500,0.000050,0.0853,0.335,96.147,audio_features,7BxsNJxh4rwA4a2QPuHKVq,spotify:track:7BxsNJxh4rwA4a2QPuHKVq,https://api.spotify.com/v1/tracks/7BxsNJxh4rwA...,https://api.spotify.com/v1/audio-analysis/7Bxs...,176815,3
6,0.744,0.22200,2,-12.511,1,0.1760,0.82100,0.000046,0.1090,0.745,80.453,audio_features,2hQRmJmTsYvRteN1QYlSkX,spotify:track:2hQRmJmTsYvRteN1QYlSkX,https://api.spotify.com/v1/tracks/2hQRmJmTsYvR...,https://api.spotify.com/v1/audio-analysis/2hQR...,244893,4
7,0.430,0.24000,8,-10.561,1,0.0526,0.73700,0.006160,0.0924,0.367,98.009,audio_features,33HqJZ1fR5helbqocrTRyK,spotify:track:33HqJZ1fR5helbqocrTRyK,https://api.spotify.com/v1/tracks/33HqJZ1fR5he...,https://api.spotify.com/v1/audio-analysis/33Hq...,232280,4
8,0.447,0.58300,8,-12.031,1,0.0360,0.06880,0.000000,0.1780,0.894,167.994,audio_features,3x7oLA01UwNZWqHnX0Brgn,spotify:track:3x7oLA01UwNZWqHnX0Brgn,https://api.spotify.com/v1/tracks/3x7oLA01UwNZ...,https://api.spotify.com/v1/audio-analysis/3x7o...,150160,4
9,0.707,0.60900,9,-10.549,0,0.0571,0.36700,0.000005,0.2040,0.846,129.145,audio_features,6jiUhLBjMV8ve9eORFDsi6,spotify:track:6jiUhLBjMV8ve9eORFDsi6,https://api.spotify.com/v1/tracks/6jiUhLBjMV8v...,https://api.spotify.com/v1/audio-analysis/6jiU...,169547,4


In [335]:
for i in french_df['key']:
    print(i)

0
6
4
2
11
1
2
8
8
9
9
0
7
11
1
8
1
1
5
9
9
4
4
0
1
5
7
11
4
0
2
1
0
4
2
11
6
3
11
11
9
4
11
0
7
4
7
4
9
11
11
10
2


## Call Functions 1.4

In [42]:
# retrieving track titles from playlist
def fetch_titles(songs):
    titles = []
    for song in songs['items']:
        titles.append(song['track']['name'])
    return titles

In [49]:
# calls playlist, returns titles of songs in playlist as dataframe
def fetch_track_names(genre_id):
    songs = call_spotify(genre_id)
    titles = fetch_titles(songs)
    df = pd.DataFrame(titles)
    df = df.rename(columns={0: 'Song Titles'})
    return df

In [50]:
# example of retrieving all track names
fetch_track_names(genre_dict.get('hip hop playlist'))

,Song Titles
0,ROXANNE
1,Good News
2,NIGHTTIME (Interlude)
3,Old Town Road (feat. Billy Ray Cyrus) - Remix
4,Lalala
...,...
95,A Thousand Bad Times
96,Candy (feat. Trippie Redd)
97,By Mistake (Remix) (feat. Juicy J & Project Pat)
98,Flowers


## Call Functions 1.5

In [59]:
# retrieving track tempos 
def get_tempo(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_tempo = r_j['track']['tempo']
    return track_tempo

In [283]:
# retrieving all track tempos
def get_all_tempos(genre_id):
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    tempos = []
    for i in tqdm(ids):
        tempos.append(get_tempo(i))
    return tempos

In [60]:
# retrieving track tempo confidence
def get_tempo_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    tempo_confidence = r_j['track']['tempo_confidence']
    return tempo_confidence

In [272]:
# convert key number to pitch name
def convert_key_to_pitchname(track_key):
    if track_key == -1:
        return 'Key Not Defined'
    if track_key == 0:
        return 'C'
    if track_key == 1:
        return 'C#/Db'
    if track_key == 2:
        return 'D'
    if track_key == 3:
        return 'D#/Eb'
    if track_key == 4:
        return 'E'
    if track_key == 5:
        return 'F'
    if track_key == 6:
        return 'F#/Gb'
    if track_key == 7:
        return 'G'
    if track_key == 8:
        return 'G#/Ab'
    if track_key == 9:
        return 'A'
    if track_key == 10:
        return 'A#/Bb'
    if track_key == 11:
        return 'B'

In [273]:
# retrieving track key
def get_key(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_key = r_j['track']['key']
    converted_key = convert_key_to_pitchname(track_key)
    return converted_key

In [350]:
# retrieving all track keys from playlist
def get_all_keys(genre_id):
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    keys = []
    count = 0
    for i in ids:
        keys.append(get_key(i))
        print('--- key data {} retrieved ---'.format(count))
        count += 1
    return keys
# french_pl_key_df = pd.DataFrame(get_all_keys(genre_dict.get('french playlist')))

In [78]:
# retrieving track key confidence
def get_key_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    key_confidence = r_j['track']['key_confidence']
    return key_confidence

In [281]:
# convert mode to diatonic name
def convert_mode_to_diatonic_name(track_mode):
    if track_mode == -1:
        return 'Mode Not Defined'
    if track_mode == 0:
        return 'Minor'
    if track_mode == 1:
        return 'Major'

In [282]:
# retrieving track mode of key (maj or min)
def get_key_mode(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id
    r = requests.get(link, headers=headers)
    r_j = r.json()
    track_mode = r_j['track']['mode']
    converted_mode = convert_mode_to_diatonic_name(track_mode)
    return converted_mode

In [349]:
# retrieving all track key modes from playlist
def get_all_key_modes(genre_id):
    songs = call_spotify(genre_id)
    ids = fetch_ids(songs)
    modes = []
    count = 0
    for i in ids:
        modes.append(get_key_mode(i))
        print('--- mode data {} retrieved ---'.format(count))
        count += 1
    return modes
# french_pl_km_df = pd.DataFrame(get_all_key_modes(genre_dict.get('french playlist')))

In [80]:
# retrieveing track key mode confidence
def get_mode_consistency(song_id):
    base = 'https://api.spotify.com/v1/audio-analysis/'
    link = base + song_id 
    r = requests.get(link, headers=headers)
    r_j = r.json()
    mode_confidence = r_j['track']['mode_confidence']
    return mode_confidence

## Feature Engineering
In order to further describe how each genre might differ other than through the high level dimensions offered by Spotify's API, below are engineered features to help better describe the form of a track within a genre. The three elements of form explored are: 
    1. Harmonic Progression
    2. Modal Progression
    3. Tempo Progression
    
 These features are engineered using the audio analysis end points. Diving into the segment anaylsis and stringing each segment together to recognize the element's progression through out the track. 

In [110]:
# get audio analysis links
def audio_analysis_links(song_ids):
    base =  'https://api.spotify.com/v1/audio-analysis/'
    list_of_audio_analysis_links = []
    for i in tqdm(song_ids):
        list_of_audio_analysis_links.append(base + str(i))
    return list_of_audio_analysis_links

In [88]:
# example of retriving all audio anaylsis links for each song in playlist
# audio_analysis_links(fetch_ids(call_spotify(genre_dict.get('hip hop playlist'))))

['https://api.spotify.com/v1/audio-analysis/696DnlkuDOXcMAnKlTgXXK',
 'https://api.spotify.com/v1/audio-analysis/3ClBKQkKoaUQ6UOhe2xlJK',
 'https://api.spotify.com/v1/audio-analysis/3X6KN4WFgPYBETFMMX675P',
 'https://api.spotify.com/v1/audio-analysis/6u7jPi22kF8CTQ3rb9DHE7',
 'https://api.spotify.com/v1/audio-analysis/51Fjme0JiitpyXKuyQiCDo',
 'https://api.spotify.com/v1/audio-analysis/6fTt0CH2t0mdeB2N9XFG5r',
 'https://api.spotify.com/v1/audio-analysis/4VginDwYTP2eaHJzO0QMjG',
 'https://api.spotify.com/v1/audio-analysis/1iSqfoUFnQwV0QW1EfUit8',
 'https://api.spotify.com/v1/audio-analysis/56sxR3i2zr7AtFbgGu12ZD',
 'https://api.spotify.com/v1/audio-analysis/3VyjsVV24RmBIbWJAeUJNu',
 'https://api.spotify.com/v1/audio-analysis/5E1jEFXElio2eOxjJ3WLnk',
 'https://api.spotify.com/v1/audio-analysis/6MWtB6iiXyIwun0YzU6DFP',
 'https://api.spotify.com/v1/audio-analysis/7aiClxsDWFRQ0Kzk5KI5ku',
 'https://api.spotify.com/v1/audio-analysis/1CxYDvfq3KVvsPSZ9NCdM9',
 'https://api.spotify.com/v1/audio

In [139]:
# retrieve dictionary of track analysis data
def fetch_analysis(links):
    analysis_data = []
    count = 0
    for i in links:
        r = requests.get(i, headers=headers)
        r_j = r.content
        analysis_data.append(json.loads(r_j))
        print('--- data on link {} retrieved--'.format(count))
        count += 1
    return analysis_data


In [94]:
# fetch_analysis(audio_analysis_links(fetch_ids(call_spotify(genre_dict.get('hip hop playlist')))))

In [191]:
# keys of section (chord progression) in track
def get_harmonic_progression(song):
    section_keys = []
    for section in song['sections']:
        k = section['key']
        if k == -1:
            section_keys.append('Key not defined')
        if k == 0:
            section_keys.append('C')
        if k == 1:
            section_keys.append('C#/Db')
        if k == 2:
            section_keys.append('D')
        if k == 3:
            section_keys.append('D#/Eb')
        if k == 4: 
            section_keys.append('E')
        if k == 5: 
            section_keys.append('F')
        if k == 6:
            section_keys.append('F#/Gb')
        if k == 7:
            section_keys.append('G')
        if k == 8:
            section_keys.append('G#/Ab')
        if k == 9:
            section_keys.append('A')
        if k == 10:
            section_keys.append('A#/Bb')
        if k == 11: 
            section_keys.append('B')
    return section_keys

In [192]:
# mode of key in section (tonal progression)    
def get_harmonic_mode_progression(song):   
    section_key_modes = []
    for section in song['sections']:
        m = section['mode']
        if m == -1:
            section_key_modes.append('Mode not Defined')
        if m == 0:
            section_key_modes.append('Minor')
        if m == 1:
            section_key_modes.append('Major')
    return section_key_modes

In [193]:
# tempo changes through out sections
def get_tempo_progression(song):    
    section_tempo = []
    for section in song['sections']:
        section_tempo.append(section['tempo'])
    return section_tempo

In [194]:
# returns section info on track passed through
def get_section_info(songs):
    section_list = []
    for song in tqdm(songs):
        h_prog = get_harmonic_progression(song)
        m_prog = get_harmonic_mode_progression(song)
        t_prog = get_tempo_progression(song)
        section_prog = [h_prog, m_prog, t_prog]
        section_list.append(section_prog)
    return section_list

In [358]:
# retrieve all converted track key and key mode from playlist
def get_track_global_key_modes(genre_id):
    print('Getting Track Keys')
    track_keys = get_all_keys(genre_id)
    print('Getting Track Modes')
    track_modes = get_all_key_modes(genre_id)
    print('Compiling Global Data')
    print('\n')
    harmonic_info = ([track_modes track_keys])
    return harmonic_info

## Function to Output all Engineered Features

In [306]:
# calls spotify playlist, returns section info on each track in playlist
def fetch_sections_info(genre_id):
    print('Getting Section Data')
    print('\n')
    data = call_spotify(genre_id)
    ids = fetch_ids(data)
    links = audio_analysis_links(ids)
    songs = fetch_analysis(links)
    section_info = get_section_info(songs)
    return section_info

In [307]:
# combine global and section info of each track from playlist 
# NOTE: for this function to work -- it needs a pivot table - or make function the cell below
def get_global_and_section_info(genre_id):
    print('Getting Global Data')
    print('\n')
    global_section_info = []
    global_info = get_track_global_key_modes(genre_id)
    print('Global Data Retrieved')
    print('\n')
    print('Getting Section Data')
    section_info = fetch_sections_info(genre_id)
    print('Section Data Retrieved')
    global_section_info = [global_info, section_info]
    print('Global and Section Data Compiled')
    return global_section_info
    

## Example of Engineered Features

In [364]:
french_pl_keys_df = pd.DataFrame(get_all_keys(genre_dict.get('french playlist')))
french_pl_keys_df = french_pl_keys_df.rename(columns={0: 'Track Keys'})
french_pl_key_modes_df = pd.DataFrame(get_all_key_modes(genre_dict.get('french playlist')))
french_pl_key_modes_df = french_pl_key_modes_df.rename(columns={0: 'Track Modes'})
french_global = pd.concat([french_pl_key_modes_df, french_pl_keys_df], axis = 1)

french_sections_df = pd.DataFrame(fetch_sections_info(genre_dict.get('french playlist')))
french_sections_df = french_sections_df.rename(columns={0: 'Harmonic Progression', 
                                                          1: 'Modal Progression',
                                                          2: 'Tempo Progression'})

french_global_and_section_info_df = pd.concat([french_global, french_sections_df], axis = 1)

--- key data 0 retrieved ---
--- key data 1 retrieved ---
--- key data 2 retrieved ---
--- key data 3 retrieved ---
--- key data 4 retrieved ---
--- key data 5 retrieved ---
--- key data 6 retrieved ---
--- key data 7 retrieved ---
--- key data 8 retrieved ---
--- key data 9 retrieved ---
--- key data 10 retrieved ---
--- key data 11 retrieved ---
--- key data 12 retrieved ---
--- key data 13 retrieved ---
--- key data 14 retrieved ---
--- key data 15 retrieved ---
--- key data 16 retrieved ---
--- key data 17 retrieved ---
--- key data 18 retrieved ---
--- key data 19 retrieved ---
--- key data 20 retrieved ---
--- key data 21 retrieved ---
--- key data 22 retrieved ---
--- key data 23 retrieved ---
--- key data 24 retrieved ---
--- key data 25 retrieved ---
--- key data 26 retrieved ---
--- key data 27 retrieved ---
--- key data 28 retrieved ---
--- key data 29 retrieved ---
--- key data 30 retrieved ---
--- key data 31 retrieved ---
--- key data 32 retrieved ---
--- key data 33 retr






100%|██████████| 53/53 [00:00<00:00, 408636.24it/s][A


--- data on link 0 retrieved--
--- data on link 1 retrieved--
--- data on link 2 retrieved--
--- data on link 3 retrieved--
--- data on link 4 retrieved--
--- data on link 5 retrieved--
--- data on link 6 retrieved--
--- data on link 7 retrieved--
--- data on link 8 retrieved--
--- data on link 9 retrieved--
--- data on link 10 retrieved--
--- data on link 11 retrieved--
--- data on link 12 retrieved--
--- data on link 13 retrieved--
--- data on link 14 retrieved--
--- data on link 15 retrieved--
--- data on link 16 retrieved--
--- data on link 17 retrieved--
--- data on link 18 retrieved--
--- data on link 19 retrieved--
--- data on link 20 retrieved--
--- data on link 21 retrieved--
--- data on link 22 retrieved--
--- data on link 23 retrieved--
--- data on link 24 retrieved--
--- data on link 25 retrieved--
--- data on link 26 retrieved--
--- data on link 27 retrieved--
--- data on link 28 retrieved--
--- data on link 29 retrieved--
--- data on link 30 retrieved--
--- data on link 3






100%|██████████| 53/53 [00:00<00:00, 22352.75it/s]

--- data on link 52 retrieved--


In [365]:
french_global_and_section_info_df

,Track Modes,Track Keys,Harmonic Progression,Modal Progression,Tempo Progression
0,Major,C,"[F, C, C, F#/Gb, C, C, B, A, D]","[Major, Major, Major, Minor, Major, Major, Min...","[95.392, 112.027, 122.547, 83.14, 72.227, 69.5..."
1,Minor,F#/Gb,"[D, B, F#/Gb, C#/Db, E, D, C#/Db, F#/Gb, B]","[Major, Minor, Minor, Major, Major, Major, Maj...","[138.424, 136.428, 131.483, 133.906, 124.14, 1..."
2,Major,E,"[G#/Ab, G#/Ab, G#/Ab, E, B, E, F#/Gb, E, E, G#...","[Minor, Minor, Minor, Major, Major, Major, Maj...","[88.913, 89.599, 90.038, 90.039, 89.875, 89.98..."
3,Major,D,"[D, D, A, D#/Eb, D, D#/Eb, B, D]","[Major, Major, Major, Major, Major, Major, Min...","[115.29, 116.341, 115.261, 115.403, 115.378, 1..."
4,Minor,B,"[C, F#/Gb, D, F#/Gb, F#/Gb, D, B, F#/Gb, B]","[Major, Minor, Major, Minor, Minor, Major, Min...","[101.869, 101.684, 102.357, 101.909, 100.553, ..."
5,Major,C#/Db,"[C#/Db, F, C#/Db, C#/Db, F#/Gb, F, A#/Bb, F, C...","[Major, Minor, Major, Major, Major, Minor, Min...","[97.814, 98.78, 95.667, 96.695, 97.41, 96.797,..."
6,Major,D,"[C, C, D, E, D, F#/Gb, B, D, C, D, F#/Gb, E]","[Major, Major, Major, Major, Major, Minor, Maj...","[78.914, 79.297, 81.502, 80.162, 82.397, 80.29..."
7,Major,G#/Ab,"[C, B, C#/Db, E, D#/Eb, D#/Eb, G#/Ab, G#/Ab, F...","[Minor, Major, Minor, Major, Major, Minor, Maj...","[99.943, 94.148, 96.808, 98.88, 97.04, 100.721..."
8,Major,G#/Ab,"[G#/Ab, G#/Ab, B, G#/Ab, G#/Ab, G#/Ab, D#/Eb]","[Major, Minor, Major, Major, Major, Minor, Major]","[167.717, 167.164, 168.101, 167.423, 167.588, ..."
9,Minor,A,"[C, A, C, A, A, A, C, A, D]","[Major, Minor, Major, Minor, Minor, Minor, Maj...","[126.69, 128.926, 129.333, 129.358, 129.001, 1..."


## Feature Engineering Function 1.2
Now that the dataframe is generated. There are a few more steps necessary before this data can be used.
    1. Clean dataframe cells so each cell has strings and not lists
    2. Transform the Harmonic Progression into Roman Numerial Analysis

In [366]:
# function to convert list of words into string
def l_to_s(_list):
    s = ''
    for i in _list:
        s = s + i + ','
    return s[:-2] 

In [367]:
# function to convert list of integers into a string
def l_to_int(_list):
    s = ''
    for i in _list:
        s = s + str(i) + ','
    return s[:-2] 

In [368]:
# cleaning
french_global_and_section_info_df['Harmonic Progression'] = french_global_and_section_info_df['Harmonic Progression'].apply(lambda x: list(x))
french_global_and_section_info_df['Modal Progression'] = french_global_and_section_info_df['Modal Progression'].apply(lambda x: l_to_s(x))
french_global_and_section_info_df['Tempo Progression'] = french_global_and_section_info_df['Tempo Progression'].apply(lambda x: l_to_int(x))

In [369]:
french_global_and_section_info_df.head()

,Track Modes,Track Keys,Harmonic Progression,Modal Progression,Tempo Progression
0,Major,C,"[F, C, C, F#/Gb, C, C, B, A, D]","Major,Major,Major,Minor,Major,Major,Minor,Mino...","95.392,112.027,122.547,83.14,72.227,69.553,67...."
1,Minor,F#/Gb,"[D, B, F#/Gb, C#/Db, E, D, C#/Db, F#/Gb, B]","Major,Minor,Minor,Major,Major,Major,Major,Mino...","138.424,136.428,131.483,133.906,124.14,129.318..."
2,Major,E,"[G#/Ab, G#/Ab, G#/Ab, E, B, E, F#/Gb, E, E, G#...","Minor,Minor,Minor,Major,Major,Major,Major,Majo...","88.913,89.599,90.038,90.039,89.875,89.987,92.3..."
3,Major,D,"[D, D, A, D#/Eb, D, D#/Eb, B, D]","Major,Major,Major,Major,Major,Major,Minor,Majo","115.29,116.341,115.261,115.403,115.378,115.338..."
4,Minor,B,"[C, F#/Gb, D, F#/Gb, F#/Gb, D, B, F#/Gb, B]","Major,Minor,Major,Minor,Minor,Major,Minor,Majo...","101.869,101.684,102.357,101.909,100.553,102.24..."


In [174]:
# read harmonic_function.py to see how Harmonic Progression is sorted
from harmonic_function import *

In [370]:
prog = get_progression_m(french_global_and_section_info_df['Track Modes'][4], french_global_and_section_info_df['Track Keys'][4], french_global_and_section_info_df['Harmonic Progression'][4])
prog

['bii', 'V', 'bIII', 'V', 'V', 'bIII', 'i', 'V', 'i']

In [244]:
# Next I need to convert track Key in column to a pitchname
# Followed by turning track mode in column to a Major or Minor
# Modal Progression will go from 
# Major to Major
# Major to Minor Major 
# Minor to Minor
# Minor to Major to Minor

In [ ]:
# How to deal with deal with tempo progression?
# or should it be disregarded?